In [1]:
import brightway2 as bw
from copy import deepcopy

In [2]:
bw.projects.set_current('ecoinvent 3.3 cutoff')

In [3]:
# Choose any demand 
ei = bw.Database('ecoinvent 3.3 cutoff')
demand_act = [act for act in ei if 'petroleum refinery operation' in act['name'] and 'CH' in act['location']
and 'kilowatt hour' in act['unit']][0]
method = ('IPCC 2013', 'climate change', 'GWP 100a')

In [4]:
# Save initial LCA score
lca = bw.LCA({demand_act: 1}, method)
lca.lci()
lca.lcia()
initial_score = deepcopy(lca.score)
initial_score

0.13744708018975899

### Goal: replace electricity market in demand with some other electricity market.
In the following you can see my highly inefficient implementation.

In [5]:
for exc in demand_act.exchanges():
    if 'market for electricity, medium voltage' == exc.input['name']:
        initial_electricity_market = exc.input
        break
initial_electricity_market

'market for electricity, medium voltage' (kilowatt hour, CH, None)

In [6]:
other_electricity_market = [act for act in ei if 'market for electricity, medium voltage' == act['name'] 
                            and 'DE' in act['location']][0]
other_electricity_market

'market for electricity, medium voltage' (kilowatt hour, DE, None)

In [7]:
def replace_exchange(act, act_prev, act_curr):
    '''
    act: output activity
    act_prev: exchange that needs to be deleted in `act`
    act_curr: new exchange that needs to be added instead to `act`
    '''
    for exc in act.exchanges():
        if act_prev['name'] in exc.input['name']:
            print('Replacing an exchange')
            amount = exc['amount']
            exc.delete()
            act.new_exchange(
                input = (act_curr['database'], act_curr['code']), 
                amount = amount, 
                type= "technosphere"
            ).save()
            break

In [8]:
%%time
# Replace exchange once (MC iteration) and run lca to get the score
replace_exchange(demand_act, initial_electricity_market, other_electricity_market)
lca = bw.LCA({demand_act: 1}, method)
lca.lci()
lca.lcia()
print(lca.score)

Replacing an exchange
0.14144139710362288
CPU times: user 7.32 s, sys: 445 ms, total: 7.76 s
Wall time: 7.38 s


In [9]:
%%time
# Replace it back to have the same database after all MC iterations
replace_exchange(demand_act, other_electricity_market, initial_electricity_market)
lca = bw.LCA({demand_act: 1}, method)
lca.lci()
lca.lcia()
print(lca.score)
assert(lca.score==initial_score)

Replacing an exchange
0.13744708018975899


### So we back to the initial score, but how to make it more efficient?
Is there smth like lca.redo_lci? 

also, to do the replacement of exchanges, I always need to keep track of previous and current exchanges. <br>
which is of course not nice, as this will break as soon as the user hits KeyboardInterrupt. then we would need to reimport the database, etc